In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('./data/arbres_grenoble_epsg4326(1).csv')

In [38]:
class Adder(BaseEstimator, TransformerMixin):
    def __init__(self, fill_value=0):
        """
        Initialize
        """
        self.fill_value = fill_value

    def fit(self, X, y=None):

        return self

    def transform(self, X):

        # Make a copy of the DataFrame to avoid modifying the original
        data = X.copy()

        # Remove columns with more than 50% missing values
        for col in data.columns:
            if data[col].isnull().sum() >= len(data) / 1.5:
                data = data.drop(col, axis=1)

        # Remove columns with only one unique value
        for col in data.columns:
            if len(data[col].unique()) == 1:
                data = data.drop(col, axis=1)

        # Filter outliers for numerical columns
        for col in data.select_dtypes(include=['int64', 'float64']).columns:
            Q1 = data[col].quantile(0.25)
            Q3 = data[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR

            # Replace outliers with NaN
            # data[col] = data[col].apply(lambda x: x if lower_bound <= x <= upper_bound else np.nan)

        # data['typenature'] = X['typenature']
        # data['forme'] = X['forme']
        data[['lat', 'lon']] = (
            X['geo_point_2d']
            .str.split(',', expand=True)
            .replace('', np.nan)
            .astype(float)
        )
        data = data.drop('geo_point_2d',axis=1)
        data = data.dropna(subset=['anneedeplantation'])
        data = data.drop(['code','sous_categorie_desc','code_parent_desc','bien_reference','nom'],axis=1)
#         # Preparing categorical values
#         le = LabelEncoder()
        
#         for i in data.select_dtypes(include=['object']).columns:
#             data[i] = le.fit_transform(data[i])

        return pd.DataFrame(data)

In [4]:
clean_pipe = Pipeline([
    ('clean',Adder())
])

In [5]:
X_clean = clean_pipe.fit_transform(data)
y_clean = X_clean.pop('anneedeplantation')

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=100)

In [7]:
col_cat = [col for col in X_train.columns if X_train[col].dtype == 'object']
col_num = [col for col in X_train.columns if X_train[col].dtype != 'object']

In [8]:
col_ord = ['hauteurarbre','stadededeveloppement']
col_nord = X_train.drop(columns=col_num + col_ord).columns

In [9]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
    ])

nord_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="most_frequent")),
        ('encode', OneHotEncoder(handle_unknown='ignore',sparse_output=True))
    ])

ord1_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('ord_1', OrdinalEncoder(categories=[['Moins de 10 m','de 10 m à 20 m','Plus de 20 m']])),
])

ord2_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('ord_2', OrdinalEncoder(categories=[['Arbre jeune','Arbre adulte','Arbre vieillissant']]))
])

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, col_num),
        ('ord1', ord1_pipeline,['hauteurarbre']),
        ('ord2', ord2_pipeline, ['stadededeveloppement']),
        ('nord', nord_pipeline, col_nord)
    ])

In [10]:
X_train_pip = full_pipeline.fit_transform(X_train)
X_test_pip = full_pipeline.transform(X_test)

In [11]:
X_train_pip

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 322155 stored elements and shape (23853, 1856)>

### Linear

In [66]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(X_train_pip, y_train)
y_pred = lin_reg.predict(X_test_pip)


mse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

Mean Squared Error: 7.93
R^2 Score: 0.80


### Tree

In [13]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=100)
tree_reg.fit(X_train_pip, y_train)

y_pred = tree_reg.predict(X_test_pip)

tree_mse = mean_squared_error(y_test, y_pred)
tree_rmse = np.sqrt(tree_mse)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {tree_rmse:.2f}")
print(f"R^2 Score: {r2:.2f}")

Mean Squared Error: 6.47
R^2 Score: 0.87


### Forest

In [14]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg.fit(X_train_pip, y_train)

y_pred = forest_reg.predict(X_test_pip)
forest_mse = root_mean_squared_error(y_test, y_pred)
forest_rmse = np.sqrt(forest_mse)
r2 = r2_score(y_test, y_pred)
print(f"ROOT Mean Squared Error: {forest_rmse:.2f}")
print(f"R^2 Score: {r2:.2f}")

ROOT Mean Squared Error: 2.36
R^2 Score: 0.90


In [15]:
scores = cross_val_score(forest_reg, X_train_pip, y_train,
                         scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)

print(f'min: {min(forest_rmse_scores)}')
print(f'mean: {forest_rmse_scores.mean()}')
print(f'std: {forest_rmse_scores.std()}')

min: 4.556054326635457
mean: 5.1764360728063945
std: 0.2881092101058571


In [16]:
param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'min_samples_split': [2,5,10], 'max_features': [2, 4, 6, 10], 'max_depth': [None,2,5,10]},
    # then try 6 (2×3) combinations with bootstrap set as False
    # {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True,
                           n_jobs=-1)
grid_search.fit(X_train_pip, y_train)
grid_search.best_params_

{'max_depth': None, 'max_features': 10, 'min_samples_split': 2}

In [17]:
grid_search.best_estimator_

RandomForestRegressor(max_features=10, random_state=42)

In [18]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

5.348011735352094 {'max_depth': None, 'max_features': 2, 'min_samples_split': 2}
5.63843242246716 {'max_depth': None, 'max_features': 2, 'min_samples_split': 5}
6.035640513858206 {'max_depth': None, 'max_features': 2, 'min_samples_split': 10}
5.29751750008417 {'max_depth': None, 'max_features': 4, 'min_samples_split': 2}
5.6073983502634155 {'max_depth': None, 'max_features': 4, 'min_samples_split': 5}
6.010923912373829 {'max_depth': None, 'max_features': 4, 'min_samples_split': 10}
5.28547081631759 {'max_depth': None, 'max_features': 6, 'min_samples_split': 2}
5.511380157393686 {'max_depth': None, 'max_features': 6, 'min_samples_split': 5}
5.942852320203262 {'max_depth': None, 'max_features': 6, 'min_samples_split': 10}
5.25966573745949 {'max_depth': None, 'max_features': 10, 'min_samples_split': 2}
5.454674384605326 {'max_depth': None, 'max_features': 10, 'min_samples_split': 5}
5.883886844736563 {'max_depth': None, 'max_features': 10, 'min_samples_split': 10}
17.94679893942571 {'max_

### random search

In [19]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [20]:
param_distribs = {
        'n_estimators': randint(low=1, high=30),
        'max_features': randint(low=1, high=14),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(X_train_pip, y_train)
cvres = rnd_search.cv_results_

/home/dmitrii.druzhbin@Digital-Grenoble.local/anaconda3/envs/jupyterlab3.6.7/lib/python3.11/site-packages/numpy/ma/core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [21]:
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

5.469557993282264 {'max_features': 7, 'n_estimators': 20}
5.517885698967405 {'max_features': 13, 'n_estimators': 15}
5.871267139444241 {'max_features': 11, 'n_estimators': 8}
5.42921415003971 {'max_features': 13, 'n_estimators': 21}
5.443161242129311 {'max_features': 7, 'n_estimators': 26}
5.572463869700667 {'max_features': 3, 'n_estimators': 23}
5.7041104689010815 {'max_features': 11, 'n_estimators': 11}
5.44810377818253 {'max_features': 8, 'n_estimators': 21}
5.878527228323959 {'max_features': 4, 'n_estimators': 8}
6.6169940735108055 {'max_features': 8, 'n_estimators': 3}


In [87]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances = pd.DataFrame(feature_importances).head(10)

In [30]:
best = RandomForestRegressor(max_features=10, n_estimators=100, random_state=42)

In [33]:
final_model = grid_search.best_estimator_
# final_model = best

In [36]:
final_predictions = final_model.predict(X_test_pip)
y_train_predict = final_model.predict(X_train_pip)
final_rmse = root_mean_squared_error(y_test, final_predictions)
train_rmse = root_mean_squared_error(y_train, y_train_predict)

In [37]:
print(f'Final RMSE: {final_rmse}')
print(f'Final train RMSE: {train_rmse}')

Final RMSE: 5.242409455597835
Final train RMSE: 1.8679236525971252


#### 95% confidence interval

In [28]:
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
loc=squared_errors.mean(),
scale=stats.sem(squared_errors)))

array([4.78849961, 5.66003403])

### Full pipeline with a model

In [76]:
from sklearn.ensemble import ExtraTreesRegressor

full_pipeline_with_predictor = Pipeline([
        ("num", full_pipeline),
        ('linear', ExtraTreesRegressor())
    ])

full_pipeline_with_predictor.fit(X_train, y_train)

y_pred = full_pipeline_with_predictor.predict(X_test)
y_pred_train = full_pipeline_with_predictor.predict(X_train)

test_rmse = root_mean_squared_error(y_test, y_pred)
train_rmse = root_mean_squared_error(y_train, y_pred_train)
print(f'Final RMSE: {test_rmse}')
print(f'Final train RMSE: {train_rmse}')

In [84]:
featurenames = full_pipeline.get_feature_names_out()

In [89]:
feature_importances['feature_name'] = pd.DataFrame(featurenames)

In [90]:
feature_importances

,0,feature_name
0,0.143082,num__elem_point_id
1,0.031317,num__adr_secteur
2,0.062096,num__lat
3,0.062599,num__lon
4,0.048706,ord1__hauteurarbre
5,0.039009,ord2__stadededeveloppement
6,0.002932,nord__sous_categorie_ESP065
7,0.009898,nord__sous_categorie_ESP151
8,0.009043,nord__sous_categorie_ESP174
9,0.002814,nord__sous_categorie_ESP187


In [96]:
from sklearn.ensemble import AdaBoostRegressor

full_pipeline_with_predictor = Pipeline([
        ("num", full_pipeline),
        ('linear', AdaBoostRegressor())
    ])

full_pipeline_with_predictor.fit(X_train, y_train)

y_pred = full_pipeline_with_predictor.predict(X_test)
y_pred_train = full_pipeline_with_predictor.predict(X_train)

test_rmse = root_mean_squared_error(y_test, y_pred)
train_rmse = root_mean_squared_error(y_train, y_pred_train)
print(f'Final RMSE: {test_rmse:.2f}')
print(f'Final train RMSE: {train_rmse:.2f}')

Final RMSE: 13.78
Final train RMSE: 13.74


In [103]:
from sklearn.ensemble import GradientBoostingRegressor

params = {
    'max_depth': [2,4,8,None],
    'n_estimators': [3,5,10],
    'learning_rate': [0.1,1,10]
}

grid_search = GridSearchCV(GradientBoostingRegressor(), params, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True,
                           n_jobs=-1)

full_pipeline_with_predictor = Pipeline([
        ("num", full_pipeline),
        ('linear', grid_search)
    ])

full_pipeline_with_predictor.fit(X_train, y_train)

y_pred = full_pipeline_with_predictor.predict(X_test)
y_pred_train = full_pipeline_with_predictor.predict(X_train)

test_rmse = root_mean_squared_error(y_test, y_pred)
train_rmse = root_mean_squared_error(y_train, y_pred_train)
print(f'Final RMSE: {test_rmse:.2f}')
print(f'Final train RMSE: {train_rmse:.2f}')

Final RMSE: 6.32
Final train RMSE: 0.00


In [104]:
grid_search.best_estimator_

GradientBoostingRegressor(learning_rate=1, max_depth=None, n_estimators=3)

In [111]:
import xgboost

params = {
    'max_depth': [2,4,8,None],
    'n_estimators': [3,5,10],
    'learning_rate': [0.1,0.3,1,10],
    'subsample': [1,2,5]
}

grid_search = GridSearchCV(xgboost.XGBRegressor(), params, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True,
                           n_jobs=-1)

full_pipeline_with_predictor = Pipeline([
        ("num", full_pipeline),
        ('linear', grid_search)
    ])

full_pipeline_with_predictor.fit(X_train, y_train)

y_pred = full_pipeline_with_predictor.predict(X_test)
y_pred_train = full_pipeline_with_predictor.predict(X_train)

test_rmse = root_mean_squared_error(y_test, y_pred)
train_rmse = root_mean_squared_error(y_train, y_pred_train)
print(f'Final RMSE: {test_rmse:.2f}')
print(f'Final train RMSE: {train_rmse:.2f}')